In [1]:
import pandas as pd
import openpyxl
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta
import pyodbc
import urllib.parse

### Functions

In [2]:
def load_excel(filename):
    wb = openpyxl.load_workbook(filename, read_only=True)
    ws = wb['Sheet1']
    header_row_idx = None
    for i, row in enumerate(ws.iter_rows(max_col=2, max_row=10, values_only=True)):
        if row and 'Case Number' in row:
            header_row_idx = i
            break
    wb.close()
    if header_row_idx is not None:
        df = pd.read_excel(filename, sheet_name='Sheet1', skiprows=header_row_idx)
        return df
    else:
        raise ValueError(f"Header row with 'Case Number' not found in: {filename}")
    
def convert_to_date(df):
    dtimeFields = ['Case Date', 'Case Submission Date','Latest Action Date','Transferred to Geospatial','GEO Completion','GEO S Completion','Transferred to Ops', 'Attachment Added Date', "ListDate"]
    for field in dtimeFields:
        if field in df.columns:
            df[field] = pd.to_datetime(df[field]).dt.date
    return df

### DB Configurations

In [3]:
# Define config at the top of the file
AppDB_CONFIG = {
    "server": '0003-MAAL-01\\LASSQLSERVER',
    "database": 'LASCaseWorkerApp',
    "username": 'LASCaseWorker',
    "password": 'LASCaseWorker'
}

# Utility function to create a connection
def get_connection_Sql():
    return pyodbc.connect(
        f"DRIVER={{SQL Server}};"
        f"SERVER={AppDB_CONFIG['server']};"
        f"DATABASE={AppDB_CONFIG['database']};"
        f"UID={AppDB_CONFIG['username']};"
        f"PWD={AppDB_CONFIG['password']};"
    )

## Dashboard DB SQL
DashDB_CONFIG = {
    "server": '0003-MAAL-01\\LASSQLSERVER',
    "database": 'GRSDASHBOARD',
    "username": 'lasapp',
    "password": 'lasapp@LAS123'
}

# Build ODBC connection string from existing DB_CONFIG
odbc_params = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={DashDB_CONFIG['server']};"
    f"DATABASE={DashDB_CONFIG['database']};"
    f"UID={DashDB_CONFIG['username']};"
    f"PWD={DashDB_CONFIG['password']};"
)

DashPost = {
    "server":"127.0.0.1",
    "port": '5432',
    "database": "GSA",
    "username": "postgres",
    "password": "1234"
}
## Dashboard DB PostgreSQL
connection_str_post = f"postgresql://{DashPost['username']}:{DashPost['password']}@{DashPost['server']}:{DashPost['port']}/{DashPost['database']}"

In [4]:
# odbc_connect_str = urllib.parse.quote_plus(odbc_params)
# Create SQLAlchemy engine for SQL Server via pyodbc
# engine_sqlserver = create_engine(f"mssql+pyodbc:///?odbc_connect={odbc_connect_str}", fast_executemany=True)
engine_postgres = create_engine(connection_str_post)
tables = ['ApprovedCases', 'CR_Current', 'CR_Data', 'ClassificationData', 'CurrentCases', 'EditorsList', 'GeoData', 'GeoSCompletionData', 'HistoricalData', 'MG_Current', 'MG_Data', 'OpsData', 'RejectedCancelled', 'ReturnedCases', 'SR_Current', 'SR_Data', 'ST_EditorList', 'Ticketing', 'TransferToGeoData', 'Urgent', 'VIP',]

In [5]:
# query_sql = """SELECT * FROM grsdbrd."{}" """
query_post = """SELECT * FROM public."{}" """

def join_userlist(comp_df, editorlist):
    comp_df['GEO S Completion'] = pd.to_datetime(comp_df['GEO S Completion']).dt.normalize()
    editorlist = editorlist.rename({'CaseProtalName': 'Geo Supervisor'},axis=1)
    editorlist["ListDate"] = pd.to_datetime(editorlist["ListDate"]).dt.normalize()
    comp_df = comp_df.sort_values(by=["GEO S Completion", "Geo Supervisor"])
    editorlist = editorlist.sort_values(by=["ListDate", "Geo Supervisor"])
    comp_df = pd.merge_asof(comp_df, editorlist, by="Geo Supervisor", left_on="GEO S Completion", 
                            right_on="ListDate", direction='backward')
    comp_df['GEO S Completion'] = [pd.to_datetime(i).date() for i in comp_df['GEO S Completion']]
    comp_df['ListDate'] = [pd.to_datetime(i).date() for i in comp_df['ListDate']]
    return comp_df


def generate_evaluation_sheet(engine, start_date, end_date):
    query = query_post+ """WHERE "GEO S Completion" BETWEEN '{}' AND '{}' """
    value = query.format(tables[7], str(start_date), str(end_date))
    completed = convert_to_date(pd.read_sql(value, engine))
    editorList = convert_to_date(pd.read_sql(query_post.format(tables[5]), engine_postgres))
    completed = join_userlist(completed, editorList)
    # completed = completed.dropna('Geo Supervisor Recommendation')
    return completed[~completed['Geo Supervisor Recommendation'].str.contains('يعاد')]
# end = datetime.now().date()
end = pd.to_datetime('2025-09-15').date()
start = end - timedelta(days=7)
compCases = generate_evaluation_sheet(engine_postgres,start, end)


In [72]:
field_dict = {"Procedure": "", "Recommendation": "", "Topology": "", 
                      "Completeness": "", "BlockAlignment": ""}
null_fields = [i for i in field_dict][:2]
null_fields

['Procedure', 'Recommendation']

In [7]:
geocomp = pd.read_csv(r"E:\Data\Geo S Completion.csv")

C:\Users\AyMaN\AppData\Local\Temp\ipykernel_23184\1078613638.py:1: DtypeWarning: Columns (1,7,25,60,61,62,63,64,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  geocomp = pd.read_csv(r"E:\Data\Geo S Completion.csv")


In [8]:
geocomp = convert_to_date(geocomp)
geocomp.head(5)

,Case Number,Absolute Ownership,Duplicate Case,Generated Titles,Case Submission Date,Latest Action Date,Action,Assignee,Transferred to Geospatial,Return To Geo Team,...,Region,GeoAction,Rejection,EditorName,UserID,SupervisorID,SupervisorName,GroupID,ListDate,SLA
0,FR2024453665,NaN,NaN,NaN,2024-10-04,2025-02-24,CW Pool,NaN,2025-02-24,Yes,...,Riyadh,رفض,إرفاق المؤشرات,NaN,NaN,NaN,NaN,NaN,NaT,0
1,FR2024508846,NaN,NaN,NaN,2024-11-01,2025-02-24,CW Pool,NaN,2025-02-24,Yes,...,Riyadh,تعديل بيانات وصفية,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0
2,FR2024509346,NaN,NaN,NaN,2024-11-01,2025-02-24,CW Pool,NaN,2024-12-18,Yes,...,Riyadh,تعديل أبعاد الأرض,NaN,NaN,NaN,NaN,NaN,NaN,NaT,55
3,FR2024509895,NaN,NaN,NaN,2024-11-01,2025-02-24,CW Pool,NaN,2024-12-11,Yes,...,Riyadh,رفض,طلب مسجل مسبقاً,NaN,NaN,NaN,NaN,NaN,NaT,61
4,FR2024512322,NaN,NaN,NaN,2024-11-02,2025-02-24,CW Pool,NaN,2024-12-31,Yes,...,Riyadh,رفض,طلب مسجل مسبقاً,NaN,NaN,NaN,NaN,NaN,NaT,44


In [9]:
geocomp.to_sql("GeoCompletion", engine_postgres, if_exists="replace", index=False)

420

In [15]:
supvisor_id = "RAlharthi.c"
query = """SELECT "SupervisorName" FROM "GeoCompletion" WHERE "SupervisorID"='{supvisor_id}' """

In [16]:
supervisorName = pd.read_sql(query, engine_postgres)['SupervisorName'].unique().tolist()
print(supervisorName)

[]


In [68]:
last_update = pd.read_sql("""SELECT MAX("ListDate") FROM "EditorsList" """, engine_postgres).values.tolist()[0]
str(last_update[0])
editors = pd.read_sql(f"""SELECT * FROM "EditorsList" 
                      WHERE "ListDate" = '{str(last_update[0])}' """, engine_postgres)
editors = editors[(editors['GroupID'].str.contains("Morning"))|(editors['GroupID'].str.contains("Night"))]
print(len(editors))

164


In [69]:
supervisors = editors["SupervisorName"].unique().tolist()
print(len(supervisors))

11


In [58]:
147/22


6.681818181818182

In [59]:
updatedList =  pd.read_csv(r"E:\Data\EditorsData.csv")
updatedList = convert_to_date(updatedList)
updatedList = updatedList.sort_values(by=["ListDate", "CaseProtalName"], ascending=True)
updatedList

,EditorName,CaseProtalName,UserID,SupervisorID,SupervisorName,GroupID,ListDate
153,Abbas Omar Mohamed Ali,Abbas Ali,aali.c,TMQasem.c,TAHA,QC Team,2025-08-27
191,Abdallh Magdy,Abdallah Magdy,Amagdy.c,maldaly.c,Mohammed Mustafa Al-Daly,Pod-Al-Shuhada,2025-08-27
98,Abdulaziz Al-Otaibi,Abdulaziz AlOtaibi,Aotaibi.c,Akamal.c,Ammar Kamal,Editor Night Shift,2025-08-27
14,Abdulaziz Abdullah Alhegbani,Abdulaziz Alhegbani,AAlhegbani.c,RAlharthi.c,Raseel alharthi,Editor Morning shift,2025-08-27
142,abdulaziz almutlaq,Abdulaziz Almutlaq,Aalmutlaq.c,Null,Null,Missing Parcel Team,2025-08-27
...,...,...,...,...,...,...,...
3335,Mashaael Al-Sebai,NaN,malsebai.c,Null,Null,Null,2025-11-04
3394,abdullah aqeel,NaN,Aaqeel.c,Null,Null,Subsequent Team,2025-11-04
3410,Anas Omar Alhares,NaN,aalhares.c,Null,Null,TEAMLEADER,2025-11-04
3413,Tawfiq Samir Abd ELWAHAB,NaN,ttelwahab.c,Null,Null,TRANING,2025-11-04


In [60]:
updatedList = updatedList.rename({"CaseProtalName":"CasePortalName"}, axis=1)
updatedList

,EditorName,CasePortalName,UserID,SupervisorID,SupervisorName,GroupID,ListDate
153,Abbas Omar Mohamed Ali,Abbas Ali,aali.c,TMQasem.c,TAHA,QC Team,2025-08-27
191,Abdallh Magdy,Abdallah Magdy,Amagdy.c,maldaly.c,Mohammed Mustafa Al-Daly,Pod-Al-Shuhada,2025-08-27
98,Abdulaziz Al-Otaibi,Abdulaziz AlOtaibi,Aotaibi.c,Akamal.c,Ammar Kamal,Editor Night Shift,2025-08-27
14,Abdulaziz Abdullah Alhegbani,Abdulaziz Alhegbani,AAlhegbani.c,RAlharthi.c,Raseel alharthi,Editor Morning shift,2025-08-27
142,abdulaziz almutlaq,Abdulaziz Almutlaq,Aalmutlaq.c,Null,Null,Missing Parcel Team,2025-08-27
...,...,...,...,...,...,...,...
3335,Mashaael Al-Sebai,NaN,malsebai.c,Null,Null,Null,2025-11-04
3394,abdullah aqeel,NaN,Aaqeel.c,Null,Null,Subsequent Team,2025-11-04
3410,Anas Omar Alhares,NaN,aalhares.c,Null,Null,TEAMLEADER,2025-11-04
3413,Tawfiq Samir Abd ELWAHAB,NaN,ttelwahab.c,Null,Null,TRANING,2025-11-04


In [61]:
updatedList.to_sql("EditorsList", engine_postgres, if_exists='replace', index=False)

435